# Özlem Ölçer ID: 0069217 ENGR421-HW1

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scipy.linalg as linalg
import pandas as pd


def safelog(x):
    return(np.log(x + 1e-100))

In [2]:
class_labels = np.genfromtxt("hw01_class_labels.csv")
data_points = np.char.strip(np.genfromtxt('hw01_data_points.csv',delimiter=',',dtype=str))

Creating training and test sets

In [3]:
x_train = data_points[:300,:] #training set data points
y_train = class_labels[:300]  #training set labels

x_test = data_points[300:,:] #test set data points
y_test = class_labels[300:]  #test set labels

print(x_train
     )

[['C' 'A' 'G' ... 'T' 'G' 'G']
 ['G' 'A' 'C' ... 'C' 'C' 'T']
 ['T' 'T' 'T' ... 'A' 'C' 'T']
 ...
 ['C' 'T' 'G' ... 'A' 'G' 'T']
 ['C' 'A' 'G' ... 'T' 'G' 'T']
 ['T' 'C' 'C' ... 'T' 'T' 'G']]


In [4]:
# class size
K = int(np.amax(y_train))

# character size
C = x_train.shape[1]

# training set size
N_train = y_train.shape[0]

# test set size
N_test = y_test.shape[0]


Estimating model parameters

In [5]:
def pcd_fun(pcd,letter):
    for k in range(K):
        for c in range(C):
            base_total = 0
            for n in range(N_train):
                if(x_train[n][c] == letter):
                    if (y_train[n] == (k + 1)):
                        base_total = base_total + 1      
            pcd[k][c] = round((base_total / x_train[y_train==(k+1)].shape[0]),8)
    return pcd

In [6]:
def pcd_fun_test(pcd,letter):
    for k in range(K):
        for c in range(C):
            base_total = 0
            for n in range(N_test):
                if(x_test[n][c] == letter):
                    if (y_test[n] == (k + 1)):
                        base_total = base_total + 1      
            pcd[k][c] = round((base_total / x_test[y_test==(k+1)].shape[0]),8)
    return pcd

In [7]:
pAcd = [[0 for c in range(C)] for k in range(K)]
pCcd = [[0 for c in range(C)] for k in range(K)]
pGcd = [[0 for c in range(C)] for k in range(K)]
pTcd = [[0 for c in range(C)] for k in range(K)]
class_priors = []

pAcd = pcd_fun(pAcd,'A')
pCcd = pcd_fun(pCcd,'C')
pGcd = pcd_fun(pGcd,'G')
pTcd = pcd_fun(pTcd,'T')

class_priors = np.array([np.mean(y_train == (c + 1)) for c in range(K)])

print(pAcd)
print("")
print(pCcd)
print("")
print(pGcd)
print("")
print(pTcd)
print("")
print(class_priors)

[[0.28, 0.68, 0.09333333, 0.56666667, 0.68, 0.14, 0.19333333], [0.22666667, 0.24, 0.26666667, 0.18666667, 0.20666667, 0.18, 0.26]]

[[0.4, 0.08666667, 0.01333333, 0.02, 0.12, 0.06, 0.08], [0.16, 0.23333333, 0.09333333, 0.17333333, 0.21333333, 0.26666667, 0.19333333]]

[[0.21333333, 0.09333333, 0.82666667, 0.35333333, 0.1, 0.76, 0.20666667], [0.29333333, 0.27333333, 0.22666667, 0.36, 0.2, 0.23333333, 0.2]]

[[0.10666667, 0.14, 0.06666667, 0.06, 0.1, 0.04, 0.52], [0.32, 0.25333333, 0.41333333, 0.28, 0.38, 0.32, 0.34666667]]

[0.5 0.5]


In [8]:
pAcd_test = [[0 for c in range(C)] for k in range(K)]
pCcd_test = [[0 for c in range(C)] for k in range(K)]
pGcd_test = [[0 for c in range(C)] for k in range(K)]
pTcd_test = [[0 for c in range(C)] for k in range(K)]


pAcd_test = pcd_fun_test(pAcd_test,'A')
pCcd_test = pcd_fun_test(pCcd_test,'C')
pGcd_test = pcd_fun_test(pGcd_test,'G')
pTcd_test = pcd_fun_test(pTcd_test,'T')


Confurison Matrix for Training Set

In [9]:
def score_func(dt):
    score_result = []
    for c in range(K):
        ivd = 1
        result = []
        for i in range(dt.shape[0]):
            for j in range(C):  
                if(dt[i][j] == 'A'):
                    ivd += safelog(pAcd[c][j])
                elif (dt[i][j]  == 'G'):
                    ivd += safelog(pGcd[c][j])
                elif (dt[i][j]  == 'C'):
                    ivd += safelog(pCcd[c][j])
                elif (dt[i][j]  == 'T'):
                    ivd += safelog(pTcd[c][j])
            ivd += safelog(class_priors[c])
            result.append(ivd)
            ivd = 1
        score_result.append(result)
    return score_result


In [10]:
train_scores = []
score_list = score_func(x_train)
for i in range(len(score_list[0])):
        if(score_list[0][i] > score_list[1][i]):
            train_scores.append(1)
        else:
            train_scores.append(2)

In [11]:
test_scores = []
score_list = score_func(x_test)
for i in range(len(score_list[0])):
        if(score_list[0][i] > score_list[1][i]):
            test_scores.append(1)
        else:
            test_scores.append(2)

In [12]:
confusion_matrix_train = pd.crosstab(train_scores, y_train, 
                               rownames = ["y_pred"], 
                               colnames = ["y_truth"])
print("confusion_train")
print()
print(confusion_matrix_train)

confusion_train

y_truth  1.0  2.0
y_pred           
1        145   14
2          5  136


In [13]:
confusion_matrix_test = pd.crosstab(test_scores, y_test, 
                               rownames = ["y_pred"], 
                               colnames = ["y_truth"])
print("confusion_test")
print()
print(confusion_matrix_test)

confusion_test

y_truth  1.0  2.0
y_pred           
1         48    8
2          2   42
